In [ ]:
#@title
!pip install pymysql
import pymysql #Mysql
import pandas as pd #Dataframe Operation
import numpy as np #calculation
import matplotlib.pyplot as plt #ploting
from datetime import date #date
import datetime
import seaborn as sns  #ploting
from datetime import datetime
from itertools import chain
import requests
import json
from decimal import Decimal
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime,timedelta
from google.colab import files
%load_ext google.colab.data_table

pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
e='RDS link'
u='user name'
p='password'
db='daatabase LINK'
connection=pymysql.connect(host=e,user=u,passwd=p,db=db)
cursors3=connection.cursor()
cursors3.execute("""
select
BuddyData.Zone,
sename As service_area_name,
ABS(sum(if(BuddyData.status = 1,1,0))) as FreeBuddies,
ABS(sum(if(BuddyData.status = 2,1,0)))as Offlinebuddy,
ABS(sum(if(BuddyData.status = 3,1,0))) as OnJob
from
(
	select buddy.status,service_area.name as sename,bsa.service_area_id as 'Zone'
	from buddy
	left join buddy_service_areas bsa
    on buddy.id = bsa.buddy_id
    left join user
    on user.id = buddy.user_id
    left join service_area
    on bsa.service_area_id = service_area.id

    where user.is_active=1 and user.role_id=5
)as BuddyData where zone is  not null
group by 1;

""")
rows3=cursors3.fetchall()

cursors3.close()

data3=pd.DataFrame(rows3)
arra=[]
for col in range(0,len(data3.columns)):
  arra.append(cursors3.description[col][0])

data3.columns=arra
data3.dropna(inplace=True)
Area=data3[data3['FreeBuddies']>0]
Area.reset_index(inplace=True)
cursors2=connection.cursor()
cursors2.execute("""

with monthly_users as (

   SELECT
   x.customer_id as customer_id,
   x.user_month as user_month,
   x.service_area as service_area,
   x.num_orders as num_orders,
   x.total_GMV as total_GMV,
   x.discount_order as disc_order,
   x.discount_amount as disc_amount,
   min(x.user_month) over(partition by x.customer_id) as first_month,
   TIMESTAMPDIFF(MONTH, min(x.user_month) over(partition by x.customer_id),x.user_month) as usage_month,
   TIMESTAMPDIFF(MONTH, lag(x.user_month) over(partition by x.customer_id order by x.user_month) ,x.user_month ) as months_between_last_order
   FROM

     (select
      ord.customer_id,
      DATE_FORMAT(ord.accepted_at, '%Y-%m-%1') as user_month,
      sa.name as service_area,
      count(ord.customer_id) as num_orders,
      SUM(ord.total) as total_GMV,
      count(if(ord.promo_code_id is not null, ord.customer_id, NULL)) as discount_order,
      SUM(if(ord.promo_code_id is not null, ord.discount, 0)) as discount_amount
      FROM orders as ord
      left join address a on ord.address_id = a.id
left join service_area sa on a.service_area_id = sa.id
left join organization_service_areas osa on sa.id = osa.service_area_id
left join hub h on ord.hub_id = h.id
       where ord.status=9

      group by 1) as x
)

-- Default definition of an active user is >=1 order during the period
-- New Users = FTU
-- Retained Users = RT
-- New Retained Users = RN
-- Old Retained Users = RO
-- Reactivated Users = RA

SELECT

-- Total Numbers
monthly_users.service_area,
sum(monthly_users.num_orders) as total_orders,
sum(disc_order) as promo_orders,
sum(total_GMV) as total_gmv,
sum(disc_amount) as total_discount,
count(if(monthly_users.num_orders >=1, monthly_users.customer_id, NULL) ) as total_users,

-- New Users = FTU
count(if(monthly_users.usage_month = 0, 1, NULL)) as FTU,
SUM(if(monthly_users.usage_month = 0, monthly_users.num_orders, NULL)) as FTU_orders,
sum(if(monthly_users.usage_month = 0, total_gmv, NULL)) as FTU_gmv,
sum(if(monthly_users.usage_month = 0 AND (disc_amount IS NOT NULL OR disc_amount != 0), disc_amount, NULL)) as FTU_promo
FROM monthly_users
group by 1
order by 1,2;
""")

rows2=cursors2.fetchall()
cursors2.close()

data2=pd.DataFrame(rows2)
arra1=[]
for col in range(0,len(data2.columns)):
  arra1.append(cursors2.description[col][0])
data2.columns=arra1



cursors=connection.cursor()
cursors.execute("""
select
id,
name,
count(*) as UserCount
from
(
select service_area.id,service_area.name from user
left join address
on user.id=address.user_id
left join service_area
on service_area.id=address.service_area_id
where
user.id
in
(
	select
	SUBSTRING_INDEX(SUBSTRING_INDEX(string2, ']', 1), ']', -1) AS first_name
	from
	(

		select
		SUBSTRING(rule_context, 15,1000) as string2
		from
		(
			select rule_context from promo_code  where created_at>=now()-interval 3 day
		)as a
	)as b
))as c group by 1;
  """)
rowData=cursors.fetchall()
cursors.close()

USerEliminateData=pd.DataFrame(rowData)
arra=[]
for col in range(0,len(USerEliminateData.columns)):
  arra.append(cursors.description[col][0])
USerEliminateData.columns=arra
if USerEliminateData.empty:
  print("DataFrame is null")
else:
  USerEliminateData=USerEliminateData.set_index('name')
  data2=data2.set_index('service_area')
  data2['Actual User']=data2['total_users']-USerEliminateData['UserCount']
  data2['Actual User'].fillna(data2['total_users'], inplace=True)


     |████████████████████████████████| 43 kB 2.4 MB/s 
DataFrame is null


In [ ]:
data2

,service_area,total_orders,promo_orders,total_gmv,total_discount,total_users,FTU,FTU_orders,FTU_gmv,FTU_promo
0,Ajwa Supermarket - DHA PH 6,3292,2325,1409374.0,329587.0,305,305,3292,1409374.0,329587.0
1,Buffer Zone 1,58,56,25437.0,9772.0,21,21,58,25437.0,9772.0
2,Buffer Zone 2,1601,1482,802302.0,259891.5,201,201,1601,802302.0,259891.5
3,Bufferzone - Al-Wardah Supermarket,4040,3378,2325144.0,575932.5,700,700,4040,2325144.0,575932.5
4,Clifton - Shop N Save Mart & Friendship Pharmacy,951,574,434320.0,91633.5,178,178,951,434320.0,91633.5
5,Defence Phase View,335,243,132709.0,31573.0,145,145,335,132709.0,31573.0
6,DHA - Phase 6 & 7,1436,987,628537.0,156031.5,318,318,1436,628537.0,156031.5
7,DHA - Phase 6 & 8,5443,3705,2597097.0,557481.5,747,747,5443,2597097.0,557481.5
8,DHA - Shell Creek Vista,25,19,11575.0,3176.0,9,9,25,11575.0,3176.0
9,DHA - Total Pump Kh Shujaat,428,340,214536.0,59348.0,67,67,428,214536.0,59348.0


In [ ]:
Area

,index,Zone,service_area_name,FreeBuddies,Offlinebuddy,OnJob
0,0,11,Shell - Clifton Marine,5,30,5
1,1,10028,Shell Pearl - Korangi Road,5,30,5
2,2,10040,Shell Sunset Blvd,5,31,5
3,3,10042,Defence Phase View,4,28,5
4,4,10048,DHA - Phase 6 & 8,4,30,7
5,5,10049,DHA Badar & Clifton,4,29,6
6,6,10064,Ajwa Supermarket - DHA PH 6,3,16,5
7,7,10063,Gulshan Munchies Warehouse,3,20,9
8,8,10043,Ranchorline,3,44,36
9,9,10044,Garden Area,2,47,40


In [ ]:
Service_Area_Id = '10069' #@param {type:"string"}
No_Of_Users = '500' #@param {type:"string"}



In [ ]:
USerEliminateData
#@title
if Service_Area_Id=='None' or Service_Area_Id =='none':
  cursors2=connection.cursor()
  cursors2.execute("""
  select u.id as ID,u.name as Name,u.phone as Phone,u.email as Email,'Karachi' as City,'Active' as Status from user u
  where not exists (select a.user_id from address a where u.id=a.user_id)
  and u.role_id=3
  and u.phone_verified=1 and  CONVERT_TZ(u.created_at, '+00:00', '+5:00')<  now() - interval 72 hour order by id desc Limit 0,{0} ;""".format(No_Of_Users))
  rows2=cursors2.fetchall()
  cursors2.close()

  data2=pd.DataFrame(rows2)
  arra1=[]
  for col in range(0,len(data2.columns)):
    arra1.append(cursors2.description[col][0])
  data2.columns=arra1
  df=data2
  df=df.reset_index(drop=True)
  str1=Service_Area_Id
  df.to_csv(str1+'.csv')
  files.download(str1+".csv")

else:
  area_name = Service_Area_Id.split(",")
  i=0
  for l in range(0,len(area_name)):
    tempStr=area_name[i]
    i=i+1
    cursors2=connection.cursor()
    cursors2.execute("""

    select
    orders.id,orders.status as order_status, orders.accepted_at ,
    orders.customer_id, service_area.id as area_id,user.name as Name,user.phone as Phone,
    orders.total,CONVERT_TZ(orders.accepted_at, '+00:00', '+5:00') As accepted_at_P,user.email as Email,'Karachi' as City,'Active'as Status
    from orders
    left join address
    on orders.address_id=address.id
    left join service_area
    on address.service_area_id=service_area.id
    left join user
    on user.id =orders.customer_id
    where is_order_fake=0 and orders.status=9 and service_area.id={0};""".format(tempStr))

    rows2=cursors2.fetchall()
    cursors2.close()

    data2=pd.DataFrame(rows2)
    arra1=[]
    for col in range(0,len(data2.columns)):
      arra1.append(cursors2.description[col][0])
    data2.columns=arra1

    cursors2=connection.cursor()
    cursors2.execute("""
    select
    user.id as customer_id,user.name as name_y,user.phone as phone_x,CONVERT_TZ(user.created_at, '+00:00', '+5:00') As created_at_p
    ,user.email as Email,'Karachi' as City,'Active' as Status
    from user
    inner join address
    on user.id=address.user_id
    inner join service_area
    on address.service_area_id=service_area.id
    where user.role_id=3 and user.id not in (select customer_id from orders) and CONVERT_TZ(user.created_at, '+00:00', '+5:00')<  now() - interval 24 hour and service_area.id={0};""".format(tempStr))
    rows2=cursors2.fetchall()
    cursors2.close()

    user_never_order=pd.DataFrame(rows2)
    arra1=[]
    for col in range(0,len(user_never_order.columns)):
      arra1.append(cursors2.description[col][0])

    user_never_order.columns=arra1

    user_never_order['Time']=datetime.now()-user_never_order['created_at_p']

    user_never_order['percential']=user_never_order['Time'].rank(pct=True,ascending=False)
    user_never_order.sort_values('percential',inplace=True,ascending=False)

    #user_never_order
    complete=data2[data2['order_status']==9]
    complete.dropna(axis=0,subset=['accepted_at'],inplace=True)
    max_date = max(complete['accepted_at_P'])
    complete['Time']=max_date-complete['accepted_at_P']
    User_recency=complete.loc[complete.groupby("customer_id")["Time"].idxmin()]
    User_recency=User_recency[User_recency['Time']>'3 days']
    fisrt_thirty_data=User_recency[User_recency['Time']<='30 days']
    fisrt_thirty_data['Percential']=fisrt_thirty_data['Time'].rank(pct=True)
    fisrt_thirty_data['rececy_Percential']=fisrt_thirty_data['Time'].rank(pct=True)
    #Frequency of all user:
    complete2=data2[data2['order_status']==9]
    complete2.dropna(axis=0,subset=['accepted_at'],inplace=True)
    complete2['freq'] = complete2.groupby('customer_id')['total'].transform('count')
    frquency_data=complete2
    frquency_data.drop_duplicates(subset ="customer_id",keep = 'first', inplace = True)
    frquency_data.sort_values('freq',inplace=True)
    recency_frequecy_data = pd.merge(left=fisrt_thirty_data, right=frquency_data, how='inner', left_on='customer_id', right_on='customer_id')
    recency_frequecy_data['Percential_freq']=recency_frequecy_data['freq'].rank(pct=True,ascending=False)
    recency_frequecy_data['reatation Time']=recency_frequecy_data['Time']
    recency_frequecy_data['reatation Percential']=recency_frequecy_data['Percential']
    recency_frequecy_data['freq User']=recency_frequecy_data['freq']
    recency_frequecy_data['frequency percential']=recency_frequecy_data['Percential_freq']
    recency_frequecy_data['Final Percential']=recency_frequecy_data['reatation Percential']*0.8+recency_frequecy_data['frequency percential']*0.20
    recency_frequecy_data.sort_values('Final Percential',inplace=True,ascending=False)
    above_thirty=User_recency[User_recency['Time']>'30 days']
    above_thirty['Percential']=above_thirty['Time'].rank(pct=True,ascending=False)
    reactivation_frequency = pd.merge(left=above_thirty, right=frquency_data, how='inner', left_on='customer_id', right_on='customer_id')
    reactivation_frequency['Percential_freq']=reactivation_frequency['freq'].rank(pct=True,ascending=True)
    reactivation_frequency['reatation Time']=reactivation_frequency['Time']
    reactivation_frequency['reatation Percential']=reactivation_frequency['Percential']
    reactivation_frequency['freq User']=reactivation_frequency['freq']
    reactivation_frequency['frequency percential']=reactivation_frequency['Percential_freq']
    reactivation_frequency['Final Percential']=reactivation_frequency['reatation Percential']*0.50+reactivation_frequency['frequency percential']*0.50
    reactivation_frequency.sort_values('Final Percential',inplace=True,ascending=False)
    recency_frequecy_data.reset_index(inplace=True)
    reactivation_frequency.reset_index(inplace=True)

    finalData = pd.merge(left=reactivation_frequency, right=recency_frequecy_data, how='inner', left_on='customer_id', right_on='customer_id')
    reactivation=reactivation_frequency[['customer_id','Name_x','Phone_x','Email_x','City_x','Status_x']]
    retention=recency_frequecy_data[['customer_id','Name_x','Phone_x','Email_x','City_x','Status_x']]
    New_user=user_never_order[['customer_id','name_y' ,'phone_x','Email','City','Status']]
    New_user = New_user.rename({'name_y': 'Name_x', 'phone_x': 'Phone_x','Email':'Email_x','City':'City_x','Status':'Status_x'}, axis=1)  # new method

    #New_user.columns={'customer_id','phone_x'}

    reactivation_frequency.sort_values('Final Percential',inplace=True,ascending=False)
    user_never_order.sort_values('percential',inplace=True,ascending=False)
    recency_frequecy_data.sort_values('Final Percential',inplace=True,ascending=False)

    df=pd.DataFrame()
    users=int(int(No_Of_Users)*0.33)
    temp=0
    check1=0
    check2=0
    check3=0
    check4=0
    check5=0

    if((retention.shape[0]<reactivation.shape[0]) & (retention.shape[0]<New_user.shape[0])):
      check1=1
    if((retention.shape[0]>reactivation.shape[0]) & (reactivation.shape[0]<New_user.shape[0])):
      check2=1
    if((New_user.shape[0]<reactivation.shape[0]) & (New_user.shape[0]<retention.shape[0])):
      check3=1

    if(check1):
      df=df.append(retention.head(users))
      if retention.shape[0]<users:
        temp=users-retention.shape[0]
      if reactivation.shape[0]<New_user.shape[0]:
        df=df.append(reactivation.head(users+temp))
        if reactivation.shape[0]<users+temp:
          temp=users+temp-reactivation.shape[0]
        df=df.append(New_user.head(users+temp))
      else :
        df=df.append(New_user.head(users+temp))
        if New_user.shape[0]<users+temp:
          temp=users+temp-New_user.shape[0]
        df=df.append(reactivation.head(users+temp))



    if(check2):
      df=df.append(reactivation.head(users))
      if reactivation.shape[0]<users:
        temp=users-reactivation.shape[0]
      if retention.shape[0]<New_user.shape[0]:
        df=df.append(retention.head(users+temp))
        if retention.shape[0]<users+temp:
          temp=users+temp-retention.shape[0]
        df=df.append(New_user.head(users+temp))
      else :
        df=df.append(New_user.head(users+temp))
        if New_user.shape[0]<users+temp:
          temp=users+temp-New_user.shape[0]
        df=df.append(retention.head(users+temp))

    if(check3):
      df=df.append(New_user.head(users))
      if New_user.shape[0]<users:
        temp=users-New_user.shape[0]
      if reactivation.shape[0]<retention.shape[0]:
        df=df.append(reactivation.head(users+temp))
        if reactivation.shape[0]<users+temp:
          temp=users+temp-reactivation.shape[0]
        df=df.append(retention.head(users+temp))
      else :
        df=df.append(retention.head(users+temp))
        if retention.shape[0]<users+temp:
          temp=users+temp-retention.shape[0]
        df=df.append(reactivation.head(users+temp))

    print(df.shape)
    cursors=connection.cursor()
    cursors.execute("""

    select
    SUBSTRING_INDEX(SUBSTRING_INDEX(string2, ']', 1), ']', -1) AS first_name
    from
    (

    select
    SUBSTRING(rule_context, 15,1000) as string2
    from
    (
      select rule_context from promo_code  where created_at>=now()-interval 3 day
    )as a
    )as b;
    """)
    rowData=cursors.fetchall()
    cursors.close()

    USerEliminateData=pd.DataFrame(rowData)
    arra=[]
    for col in range(0,len(USerEliminateData.columns)):
      arra.append(cursors.description[col][0])
    USerEliminateData.columns=arra

    if USerEliminateData.empty:
        print("DataFrame is null")
    else:
      customer_data=pd.DataFrame()
      for x in range(0,len(USerEliminateData.index)):
        customer_data=customer_data.append(USerEliminateData['first_name'][x].split(','))
      customer_data=customer_data.drop_duplicates()
      customer_data=customer_data.rename(columns={0:'customer_id'})
      customer_data.dropna(inplace=True)
      df = df[~df['customer_id'].isin(customer_data['customer_id'])]
      df=df.reset_index(drop=True)
      df.columns=[['ID','Name','Phone','Email','City','Status']]
      df.to_csv(tempStr+'.csv')
      files.download(tempStr+".csv")





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

(470, 6)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>